#

# Credit Bureau Data Processing Pipeline

## 1. Setup and Imports

In [77]:
!pip install category_encoders fuzzywuzzy

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')

## 2. Utility Functions

In [79]:
def detect_outliers_iqr(df, cols):
    """Detect outliers using IQR method"""
    for col in cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]

        print(f"Column: {col}")
        print(f" - Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}")
        print(f" - Outliers: {len(outliers)}\n")


def clip_outliers(df, cols, lower_pct=0.01, upper_pct=0.99):
    """Clip outliers based on percentiles"""
    df_copy = df.copy()
    for col in cols:
        lower = df_copy[col].quantile(lower_pct)
        upper = df_copy[col].quantile(upper_pct)
        df_copy[col] = df_copy[col].clip(lower, upper)
    return df_copy


def parse_money(value):
    """Convert currency strings to float"""
    if pd.isna(value):
        return np.nan
    s = re.sub(r'[^0-9.-]', '', str(value).strip())
    if s in ['', '-', '--']:
        return np.nan
    try:
        return float(s)
    except:
        return np.nan

## 3. Metadata Pipeline

In [80]:
class MetadataPipeline:
    def __init__(self, filepath):
        self.data = pd.read_csv(filepath)

    def encode_categoricals(self):
        le = LabelEncoder()
        self.data['account_status_code'] = le.fit_transform(self.data['account_status_code'])
        return self

    def create_features(self):
        current_year = datetime.now().year

        self.data['login_to_calls_ratio'] = (
            self.data['num_login_sessions'] / (self.data['num_customer_service_calls'] + 1)
        )
        self.data['account_age'] = current_year - self.data['account_open_year']
        self.data['high_activity_flag'] = (self.data['num_login_sessions'] > 18).astype(int)

        return self

    def create_time_features(self):
        def hour_to_period(hour):
            if 0 <= hour < 6:
                return 'night'
            elif 6 <= hour < 12:
                return 'morning'
            elif 12 <= hour < 18:
                return 'afternoon'
            else:
                return 'evening'

        self.data['application_period'] = self.data['application_hour'].apply(hour_to_period)

        self.data['login_activity_level'] = pd.cut(
            self.data['num_login_sessions'],
            bins=[-1, 10, 18, self.data['num_login_sessions'].max()],
            labels=['low', 'medium', 'high']
        )
        median_calls = self.data['num_customer_service_calls'].median()
        self.data['calls_level'] = (
            self.data['num_customer_service_calls'] > median_calls
        ).map({True: 'high', False: 'low'})

        return self

    def encode_and_finalize(self):
        # Keep string versions before one-hot encoding
        self.data['login_activity_level'] = self.data['login_activity_level'].astype(str)
        self.data['calls_level'] = self.data['calls_level'].astype(str)

        # One-hot encode categorical features (creates duplicates with .1 suffix)
        one_hot_cols = ['preferred_contact', 'application_period',
                       'login_activity_level', 'calls_level']

        df_onehot = pd.get_dummies(self.data[one_hot_cols], drop_first=False)

        # Target encode account_status_code
        target_enc = ce.TargetEncoder(cols=['account_status_code'])

        # Drop columns but KEEP login_activity_level and calls_level as strings
        cols_to_drop = ['application_hour', 'account_open_year',
                       'preferred_contact', 'account_status_code',
                       'random_noise_1', 'application_period',
                       'referral_code', 'application_id']

        self.data = pd.concat([
            self.data.drop(columns=cols_to_drop, errors='ignore'),
            df_onehot
        ], axis=1)

        # Convert boolean columns to int
        bool_cols = self.data.select_dtypes(include=['bool']).columns
        self.data[bool_cols] = self.data[bool_cols].astype(int)
        self.data['login_activity_level'] = pd.cut(
          self.data['num_login_sessions'],
          bins=[-1, 10, 18, self.data['num_login_sessions'].max()],
          labels=['low', 'medium', 'high']
        )
        self.data.drop(["num_login_sessions"], axis=1, inplace=True)

        # 3️⃣ Bin num_customer_service_calls into low/high
        # Any user with more than median calls can be high
        median_calls = self.data['num_customer_service_calls'].median()
        self.data['calls_level'] = (self.data['num_customer_service_calls'] > median_calls).map({True: 'high', False: 'low'})
        return self

    def process(self):
        return (
            self.encode_categoricals()
            .create_features()
            .create_time_features()
            .encode_and_finalize()
            .data
        )


metadata_processed = MetadataPipeline('application_metadata.csv').process()
metadata_processed.to_csv('metadata_final.csv', index=False)
print(f"Metadata processed: {metadata_processed.shape}")
print(f"Columns: {list(metadata_processed.columns)}")

Metadata processed: (10001, 22)
Columns: ['customer_ref', 'application_day_of_week', 'num_customer_service_calls', 'has_mobile_app', 'paperless_billing', 'login_to_calls_ratio', 'account_age', 'high_activity_flag', 'login_activity_level', 'calls_level', 'preferred_contact_Email', 'preferred_contact_Mail', 'preferred_contact_Phone', 'application_period_afternoon', 'application_period_evening', 'application_period_morning', 'application_period_night', 'login_activity_level_high', 'login_activity_level_low', 'login_activity_level_medium', 'calls_level_high', 'calls_level_low']


## 4. Credit History Pipeline

In [81]:
class CreditHistoryPipeline:
    def __init__(self, filepath):
        self.data = pd.read_parquet(filepath)

    def select_features(self):
        selected_cols = [
            'credit_score', 'oldest_credit_line_age', 'oldest_account_age_months',
            'num_delinquencies_2yrs', 'num_public_records', 'num_collections',
            'num_credit_accounts', 'total_credit_limit', 'num_inquiries_6mo',
            'recent_inquiry_count', 'account_diversity_index'
        ]
        self.data = self.data[selected_cols].copy()
        return self

    def handle_missing(self):
        self.data['num_delinquencies_2yrs'].fillna(1.0, inplace=True)
        return self

    def create_features(self):
        self.data['total_delinquencies'] = (
            self.data['num_delinquencies_2yrs'] +
            self.data['num_public_records'] +
            self.data['num_collections']
        )

        self.data['credit_utilization_ratio'] = (
            self.data['total_credit_limit'] / self.data['num_credit_accounts'].replace(0, 1)
        )

        self.data['recent_inquiry_ratio'] = (
            self.data['recent_inquiry_count'] / self.data['num_credit_accounts'].replace(0, 1)
        )

        self.data['total_recent_inquiries'] = (
            self.data['num_inquiries_6mo'] + self.data['recent_inquiry_count']
        )

        return self

    def clip_features(self):
        new_cols = ['total_delinquencies', 'credit_utilization_ratio', 'recent_inquiry_ratio']
        self.data = clip_outliers(self.data, new_cols)
        return self

    def process(self):
        return (
            self.select_features()
            .handle_missing()
            .create_features()
            .clip_features()
            .data
        )


credit_processed = CreditHistoryPipeline('credit_history.parquet').process()
credit_processed.to_csv('credit_history_final.csv', index=False)
print(f"Credit history processed: {credit_processed.shape}")
print(f"Columns: {list(credit_processed.columns)}")

Credit history processed: (10001, 15)
Columns: ['credit_score', 'oldest_credit_line_age', 'oldest_account_age_months', 'num_delinquencies_2yrs', 'num_public_records', 'num_collections', 'num_credit_accounts', 'total_credit_limit', 'num_inquiries_6mo', 'recent_inquiry_count', 'account_diversity_index', 'total_delinquencies', 'credit_utilization_ratio', 'recent_inquiry_ratio', 'total_recent_inquiries']


## 5. Loan Details Pipeline

In [82]:
from fuzzywuzzy import process

class LoanDetailsPipeline:
    def __init__(self, filepath):
        self.data = pd.read_excel(filepath)

    def clean_amount(self):
        self.data['loan_amount'] = self.data['loan_amount'].apply(parse_money)
        return self

    def normalize_loan_type(self):
        categories = ['personal', 'mortgage', 'credit_card']

        def map_to_category(value):
            if pd.isna(value):
                return 'personal'
            value_clean = str(value).lower().strip()
            best_match = process.extractOne(value_clean, categories)[0]
            return best_match

        self.data['loan_type_clean'] = self.data['loan_type'].apply(map_to_category)
        return self

    def create_features(self):
        self.data['secured_status'] = self.data['loan_type_clean'].apply(
            lambda x: 'secured' if x == 'mortgage' else 'unsecured'
        )

        def map_term_category(term_months):
            if term_months < 36:
                return 'short_term'
            elif term_months <= 120:
                return 'medium_term'
            else:
                return 'long_term'

        self.data['term_category'] = self.data['loan_term'].apply(map_term_category)

        # Create loan_risk_category
        def loan_risk_score(row):
            loan_type = row['loan_type_clean']
            purpose = row['loan_purpose']

            if loan_type == 'mortgage':
                return 'low_risk'
            if loan_type == 'credit_card':
                return 'high_risk'
            if purpose in ['debt_management']:
                return 'medium_high_risk'
            if purpose in ['medical', 'other']:
                return 'medium_risk'
            return 'medium_risk'

        self.data['loan_risk_category'] = self.data.apply(loan_risk_score, axis=1)

        return self

    def create_purpose_risk(self):
        # Load metadata for default column
        try:
            metadata = pd.read_csv('application_metadata.csv')
            self.data['default_temp'] = metadata['default'][:len(self.data)]
            risk_map = self.data.groupby('loan_purpose')['default_temp'].mean().to_dict()
            self.data['loan_purpose_risk'] = self.data['loan_purpose'].map(risk_map)
            self.data = self.data.drop(columns=['default_temp'])
        except:
            self.data['loan_purpose_risk'] = 0.5

        return self

    def encode_categoricals(self):
        le = LabelEncoder()
        categorical_cols = ['loan_type_clean', 'secured_status', 'term_category']

        for col in categorical_cols:
            self.data[col] = le.fit_transform(self.data[col])

        return self

    def finalize(self):
        cols_to_drop = ['loan_type', 'loan_purpose', 'origination_channel',
                       'marketing_campaign', 'loan_officer_id']
        self.data = self.data.drop(columns=cols_to_drop, errors='ignore')
        return self

    def process(self):
        return (
            self.clean_amount()
            .normalize_loan_type()
            .create_features()
            .create_purpose_risk()
            .encode_categoricals()
            .finalize()
            .data
        )


loan_processed = LoanDetailsPipeline('loan_details.xlsx').process()
loan_processed.to_csv('loan_details_final.csv', index=False)
print(f"Loan details processed: {loan_processed.shape}")
print(f"Columns: {list(loan_processed.columns)}")

Loan details processed: (10001, 10)
Columns: ['customer_id', 'loan_amount', 'loan_term', 'interest_rate', 'loan_to_value_ratio', 'loan_type_clean', 'secured_status', 'term_category', 'loan_risk_category', 'loan_purpose_risk']


## 6. Demographics Pipeline

In [83]:
class DemographicsPipeline:
    def __init__(self, filepath):
        self.data = pd.read_csv(filepath)

    def clean_income(self):
        self.data['annual_income'] = self.data['annual_income'].apply(parse_money)
        return self

    def handle_missing(self):
        self.data['employment_length'].fillna(
            self.data['employment_length'].mean(), inplace=True
        )
        self.data['num_dependents'].fillna(
            self.data['num_dependents'].mean(), inplace=True
        )
        return self

    def clip_features(self):
        numeric_cols = ['age', 'annual_income', 'employment_length', 'num_dependents']
        self.data = clip_outliers(self.data, numeric_cols)
        return self

    def normalize_employment_type(self):
        employment_mapping = {
            'full_time': ['Full-time', 'Full Time', 'FULL_TIME', 'Fulltime', 'FT'],
            'part_time': ['PART_TIME', 'Part-time', 'PT', 'Part Time'],
            'self_employed': ['SELF_EMPLOYED', 'Self Emp', 'Self Employed', 'Self-employed'],
            'contractor': ['Contractor', 'CONTRACT', 'Contract']
        }

        reverse_map = {v: k for k, lst in employment_mapping.items() for v in lst}
        self.data['employment_type_clean'] = self.data['employment_type'].map(reverse_map)
        self.data['employment_type_clean'].fillna('full_time', inplace=True)
        return self

    def encode_features(self):
        education_map = {
            'High School': 0, 'Some College': 1, 'Bachelor': 2,
            'Graduate': 3, 'Advanced': 4
        }
        self.data['education'] = self.data['education'].map(education_map)

        # One-hot encode employment_type_clean only
        df_employment = pd.get_dummies(
            self.data['employment_type_clean'],
            prefix='employment_type_clean',
            drop_first=False
        )

        # Label encode marital_status
        le_marital = LabelEncoder()
        self.data['marital_status'] = le_marital.fit_transform(self.data['marital_status'])

        self.data = pd.concat([self.data, df_employment], axis=1)
        self.data = self.data.drop(columns=['employment_type', 'employment_type_clean'])

        return self

    def process(self):
        return (
            self.clean_income()
            .handle_missing()
            .clip_features()
            .normalize_employment_type()
            .encode_features()
            .data
        )


demographics_processed = DemographicsPipeline('demographics.csv').process()
demographics_processed.drop(["employment_type_clean_contractor"], axis=1, inplace=True)
demographics_processed.to_csv('demographics_final.csv', index=False)
print(f"Demographics processed: {demographics_processed.shape}")
print(f"Columns: {list(demographics_processed.columns)}")

Demographics processed: (10001, 10)
Columns: ['cust_id', 'age', 'annual_income', 'employment_length', 'education', 'marital_status', 'num_dependents', 'employment_type_clean_full_time', 'employment_type_clean_part_time', 'employment_type_clean_self_employed']


## 7. Financial Ratios Pipeline

In [84]:
import json

class FinancialRatiosPipeline:
    def __init__(self, filepath):
        records = []
        with open(filepath, 'r') as f:
            for line in f:
                try:
                    records.append(json.loads(line.strip()))
                except:
                    continue
        self.data = pd.DataFrame(records)

    def clean_numeric_columns(self):
        for col in self.data.select_dtypes(include=['object']).columns:
            cleaned = (
                self.data[col]
                .astype(str)
                .str.replace(r'[\$,]', '', regex=True)
                .str.strip()
            )
            try:
                self.data[col] = pd.to_numeric(cleaned, errors='raise')
            except:
                pass

        return self

    def show_data(self):
      print(self.data["available_credit"])

    def remove_unwanted_columns(self):
        if 'revolving_balance' in self.data.columns:
            self.data = self.data.drop(columns=['revolving_balance'])
        if 'annual_debt_payment' in self.data.columns:
            self.data = self.data.drop(columns=['annual_debt_payment'])
        return self

    def winsorize_features(self):
        cols_to_winsorize = [
            'monthly_income', 'existing_monthly_debt', 'monthly_payment',
            'debt_to_income_ratio', 'debt_service_ratio', 'payment_to_income_ratio',
            'credit_utilization', 'credit_usage_amount', 'available_credit',
            'total_monthly_debt_payment', 'loan_to_annual_income', 'total_debt_amount'
        ]

        for col in cols_to_winsorize:
            if col in self.data.columns:
                self.data[col] = winsorize(self.data[col], limits=[0.01, 0.01])

        return self

    def create_features(self):
        self.data['income_remaining_after_debt'] = (
            self.data['monthly_income'] - self.data['existing_monthly_debt']
        )

        self.data['income_remaining_after_all_payments'] = (
            self.data['monthly_income'] - self.data['total_monthly_debt_payment']
        )

        self.data['free_cashflow_ratio'] = (
            self.data['monthly_free_cash_flow'] / self.data['monthly_income']
        )

        self.data['debt_burden_ratio'] = (
            self.data['total_debt_amount'] / (self.data['monthly_income'] * 12)
        )

        self.data['utilization_beyond_safe_zone'] = (
            self.data['credit_utilization'] > 0.3
        ).astype(int)

        self.data['high_payment_burden'] = (
            self.data['total_monthly_debt_payment'] / self.data['monthly_income'] > 0.4
        ).astype(int)

        self.data['debt_utilization_interaction'] = (
            self.data['total_debt_amount'] * self.data['credit_utilization']
        )

        self.data['payment_utilization_interaction'] = (
            self.data['monthly_payment'] * self.data['credit_utilization']
        )

        return self

    def process(self):
        return (
            self.clean_numeric_columns()
            .remove_unwanted_columns()
            .winsorize_features()
            .create_features()
            .data
        )


financial_processed = FinancialRatiosPipeline('financial_ratios.jsonl').process()
financial_processed.to_csv('financial_ratios_final.csv', index=False)
print(f"Financial ratios processed: {financial_processed.shape}")
print(f"Columns: {list(financial_processed.columns)}")


Financial ratios processed: (10001, 22)
Columns: ['cust_num', 'monthly_income', 'existing_monthly_debt', 'monthly_payment', 'debt_to_income_ratio', 'debt_service_ratio', 'payment_to_income_ratio', 'credit_utilization', 'credit_usage_amount', 'available_credit', 'total_monthly_debt_payment', 'loan_to_annual_income', 'total_debt_amount', 'monthly_free_cash_flow', 'income_remaining_after_debt', 'income_remaining_after_all_payments', 'free_cashflow_ratio', 'debt_burden_ratio', 'utilization_beyond_safe_zone', 'high_payment_burden', 'debt_utilization_interaction', 'payment_utilization_interaction']


## 8. Geographics Pipeline

In [85]:
class GeographicDataPipeline:
    def __init__(self, filepath):
        try:
            self.data = pd.read_xml(filepath)
        except Exception as e:
            print(f"Error reading XML: {e}")
            print("Attempting alternative XML parsing...")
            import xml.etree.ElementTree as ET

            tree = ET.parse(filepath)
            root = tree.getroot()

            records = []
            for record in root.findall('.//record'):
                record_dict = {}
                for child in record:
                    record_dict[child.tag] = child.text
                records.append(record_dict)

            self.data = pd.DataFrame(records)

            numeric_cols = [
                'regional_unemployment_rate', 'regional_median_income',
                'regional_median_rent', 'housing_price_index', 'cost_of_living_index'
            ]

            for col in numeric_cols:
                if col in self.data.columns:
                    self.data[col] = pd.to_numeric(self.data[col], errors='coerce')

    def remove_state(self):
        if 'state' in self.data.columns:
            self.data = self.data.drop(columns=['state'])
        return self

    def clip_outliers(self):
        numeric_cols = [
            'regional_unemployment_rate', 'regional_median_income',
            'regional_median_rent', 'housing_price_index', 'cost_of_living_index'
        ]

        for col in numeric_cols:
            if col in self.data.columns:
                lower = self.data[col].quantile(0.01)
                upper = self.data[col].quantile(0.99)
                self.data[col] = self.data[col].clip(lower, upper)

        return self

    def normalize_features(self):
        from sklearn.preprocessing import MinMaxScaler

        numeric_cols = [
            'regional_unemployment_rate', 'regional_median_income',
            'regional_median_rent', 'housing_price_index', 'cost_of_living_index'
        ]

        cols_to_normalize = [col for col in numeric_cols if col in self.data.columns]

        if cols_to_normalize:
            scaler = MinMaxScaler()
            self.data[cols_to_normalize] = scaler.fit_transform(self.data[cols_to_normalize])

        return self

    def process(self):
        return (
            self.remove_state()
            .clip_outliers()
            .normalize_features()
            .data
        )


import os
xml_file = 'geographic_data.xml'

if os.path.exists(xml_file):
    geographic_processed = GeographicDataPipeline(xml_file).process()
    geographic_processed.to_csv('geographic_data_final.csv', index=False)
    print(f"Geographic data processed: {geographic_processed.shape}")
    print(f"Columns: {list(geographic_processed.columns)}")
else:
    print(f"Warning: {xml_file} not found.")

Geographic data processed: (10001, 7)
Columns: ['id', 'regional_unemployment_rate', 'regional_median_income', 'regional_median_rent', 'housing_price_index', 'cost_of_living_index', 'previous_zip_code']


## 9 Merge data

In [98]:
class DataMerger:
    def __init__(self):
        self.metadata = pd.read_csv('metadata_final.csv')
        self.credit = pd.read_csv('credit_history_final.csv')
        self.loan = pd.read_csv('loan_details_final.csv')
        self.demographics = pd.read_csv('demographics_final.csv')
        self.financial = pd.read_csv('financial_ratios_final.csv')
        self.geographic = pd.read_csv('geographic_data_final.csv')

    def add_customer_ids(self):
        n_rows = len(self.metadata)

        for df in [self.credit, self.loan, self.financial, self.geographic]:
            if 'customer_id' not in df.columns:
                df['customer_id'] = self.metadata['customer_ref'].values[:len(df)]

        return self

    def merge_datasets(self):
        merged = self.metadata.copy()

        # Merge demographics
        merged = merged.merge(
            self.demographics,
            left_on='customer_ref',
            right_on='cust_id',
            how='left',
            suffixes=('', '_demo')
        )
        merged = merged.drop(columns=['cust_id'], errors='ignore')

        # Merge credit
        merged = merged.merge(
            self.credit,
            left_on='customer_ref',
            right_on='customer_id',
            how='left',
            suffixes=('', '_credit')
        )
        merged = merged.drop(columns=['customer_id'], errors='ignore')

        # Merge loan
        merged = merged.merge(
            self.loan,
            left_on='customer_ref',
            right_on='customer_id',
            how='left',
            suffixes=('', '_loan')
        )
        merged = merged.drop(columns=['customer_id'], errors='ignore')

        # Merge financial
        merged = merged.merge(
            self.financial,
            left_on='customer_ref',
            right_on='customer_id',
            how='left',
            suffixes=('', '_financial')
        )
        merged = merged.drop(columns=['customer_id'], errors='ignore')

        # Merge geographic
        merged = merged.merge(
            self.geographic,
            left_on='customer_ref',
            right_on='customer_id',
            how='left',
            suffixes=('', '_geo')
        )
        merged = merged.drop(columns=['customer_id'], errors='ignore')

        self.merged_data = merged
        return self

    def finalize(self):
        # Remove only duplicate columns with suffixes
        cols_to_drop = [col for col in self.merged_data.columns
                       if col.endswith(('_demo', '_credit', '_loan', '_financial', '_geo'))]

        # Drop customer_ref and cust_num but NOT login_activity_level or calls_level
        cols_to_drop.extend(['customer_ref', 'cust_num', "preferred_contact_Phone", "preferred_contact_Mail", "preferred_contact_Email", "login_activity_level", "calls_level", "loan_risk_category"])

        self.merged_data = self.merged_data.drop(columns=cols_to_drop, errors='ignore')

        return self

    def process(self):
        return (
            self.add_customer_ids()
            .merge_datasets()
            .finalize()
            .merged_data
        )


merger = DataMerger()
final_dataset = merger.process()
final_dataset.to_csv('final_merged_dataset.csv', index=False)

print("\n=== Merge Complete ===")
print(f"Final shape: {final_dataset.shape}")
print(f"Total columns: {len(final_dataset.columns)}")
print(f"Total missing values: {final_dataset.isnull().sum().sum()}")
print(f"\nFile saved: final_merged_dataset.csv")


=== Merge Complete ===
Final shape: (10001, 75)
Total columns: 75
Total missing values: 0

File saved: final_merged_dataset.csv


In [90]:
final_dataset

,application_day_of_week,num_customer_service_calls,has_mobile_app,paperless_billing,login_to_calls_ratio,account_age,high_activity_flag,login_activity_level,calls_level,application_period_afternoon,...,high_payment_burden,debt_utilization_interaction,payment_utilization_interaction,id,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,0,2,0,0,4.000000,7,0,medium,low,0,...,0,21619.6296,185.06766,100000,0.24,0.000000,0.252252,0.170732,0.060606,390
1,6,4,0,1,2.000000,4,0,low,high,0,...,0,9106.5696,57.52800,100001,0.84,0.709677,0.414414,0.926829,0.924242,292
2,4,3,1,1,1.000000,6,0,low,high,0,...,0,5886.7796,38.45400,100002,0.56,0.612903,0.558559,0.646341,0.742424,552
3,4,0,1,0,7.000000,14,0,low,low,0,...,1,9589.0975,131.40000,100003,0.20,0.903226,0.585586,0.670732,0.712121,716
4,1,0,1,0,6.000000,4,0,low,low,0,...,1,148651.7900,911.54700,100004,0.52,0.387097,0.360360,0.134146,0.257576,559
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,6,2,0,1,2.333333,5,0,low,low,0,...,1,185133.7488,1017.33924,109996,0.48,0.258065,0.306306,0.097561,0.075758,251
9997,5,3,1,1,1.500000,7,0,low,high,1,...,0,31497.9372,252.91350,109997,0.36,0.258065,0.423423,0.231707,0.363636,532
9998,0,1,1,0,2.000000,11,0,low,low,0,...,1,25039.9350,256.71525,109998,0.40,0.258065,0.423423,0.195122,0.393939,480
9999,5,0,0,0,6.000000,11,0,low,low,1,...,0,7583.2755,102.64486,109999,0.04,0.000000,0.099099,0.182927,0.242424,742


In [100]:
import pandas as pd
from lightgbm import Booster

# Load your saved model
model = Booster(model_file="lightgbm_model2.txt")

# Load test data
# test_df = pd.read_csv("test.csv")


# Predict probability of class 1
y_prob = model.predict(final_dataset)

# Apply a custom threshold
threshold = 0.65
y_pred = (y_prob > threshold).astype(int)

# Create result dataframe
submission = pd.DataFrame({
    "customer_id": final_dataset['id'],
    "prob": y_prob,
    "default": y_pred
})

submission.to_csv("result.csv", index=False)
